In [1]:
import boto3
import botocore


sns = boto3.client('sns')
sqs = boto3.client('sqs')

In [2]:
import os
account_no = os.environ['ACCOUNT_NO']

### Set Up Delivery Failure Notification

In [9]:
failureNotificationRoleArn = 'arn:aws:iam::{}:role/SNSFailureFeedback'.format(account_no)

In [10]:
# Set this to the serverless deployment stage name
stage = 'dev'
topic_arn = 'arn:aws:sns:us-east-1:' + account_no + ':t1-' + stage

In [8]:
# Modify the topic to enable delivery failure logging. This cannot be done via cloud
# formation at the moment.
tar = sns.set_topic_attributes(
    TopicArn=topic_arn,
    AttributeName='SQSFailureFeedbackRoleArn',
    AttributeValue=failureNotificationRoleArn
)
print(tar)

{'ResponseMetadata': {'RequestId': 'ea7fd474-dc99-5b9f-bf84-8d37d261ed15', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': 'ea7fd474-dc99-5b9f-bf84-8d37d261ed15', 'content-type': 'text/xml', 'content-length': '215', 'date': 'Tue, 15 Jun 2021 20:25:28 GMT'}, 'RetryAttempts': 0}}


### Set Up a Doomed SQS Subscription

This is to force delivery failures so we can detect and process those.

In [9]:
cq = sqs.create_queue(
    QueueName='sampleQueue-' + stage
)
print(cq)
queue_url = cq['QueueUrl']
queue_arn = 'arn:aws:sqs:us-east-1:' + account_no + ':sampleQueue-' + stage

{'QueueUrl': 'https://queue.amazonaws.com/427848627088/sampleQueue-dev', 'ResponseMetadata': {'RequestId': 'b6a52fa8-d6bd-5257-88c0-76ebd95d0143', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': 'b6a52fa8-d6bd-5257-88c0-76ebd95d0143', 'date': 'Tue, 15 Jun 2021 20:29:20 GMT', 'content-type': 'text/xml', 'content-length': '327'}, 'RetryAttempts': 0}}


In [11]:
sns.subscribe(
    TopicArn='arn:aws:sns:us-east-1:' + account_no + ':t1-' + stage,
    Protocol='sqs',
    Endpoint=queue_arn
)

{'SubscriptionArn': 'arn:aws:sns:us-east-1:427848627088:t1-dev:0d221450-39ee-4ef6-9178-172ac4f037b2',
 'ResponseMetadata': {'RequestId': '34ca3595-9a62-5ad6-bbd3-6cd68a695fb2',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '34ca3595-9a62-5ad6-bbd3-6cd68a695fb2',
   'content-type': 'text/xml',
   'content-length': '356',
   'date': 'Tue, 15 Jun 2021 20:30:03 GMT'},
  'RetryAttempts': 0}}

In [12]:
def allow_sns_to_write_to_sqs(topicarn, queuearn):
    policy_document = """{{
  "Version":"2012-10-17",
  "Statement":[
    {{
      "Sid":"MyPolicy",
      "Effect":"Allow",
      "Principal" : {{"AWS" : "*"}},
      "Action":"SQS:SendMessage",
      "Resource": "{}",
      "Condition":{{
        "ArnLike":{{
          "aws:SourceArn": "{}noworky"
        }}
      }}
    }}
  ]
}}""".format(queuearn, topicarn)

    return policy_document

In [13]:
#policy_json = allow_sns_to_write_to_sqs(topic_arn1, queue_arn)
policy_json = allow_sns_to_write_to_sqs('arn:aws:sns:us-east-1:' + account_no + ':*', queue_arn)

response = sqs.set_queue_attributes(
    QueueUrl = queue_url,
    Attributes = {
        'Policy' : policy_json
    }
)
print(response)

{'ResponseMetadata': {'RequestId': '7e10612f-37ef-52e3-adb4-386c27c92cbe', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '7e10612f-37ef-52e3-adb4-386c27c92cbe', 'date': 'Tue, 15 Jun 2021 20:30:30 GMT', 'content-type': 'text/xml', 'content-length': '225'}, 'RetryAttempts': 0}}


### Delivery Log Queries

In [3]:
client = boto3.client('logs')

In [11]:
import time
nowish = int(time.time())
hour_ago = nowish - (60*60)

q = client.start_query(
    logGroupName='sns/us-east-1/427848627088/t1-' + stage + '/Failure',
    startTime=hour_ago,
    endTime=nowish,
    queryString='fields @timestamp, @message | sort @timestamp desc | limit 20',
    limit=123
)

print(q)

{'queryId': '914ebb39-ca72-4833-bac2-4c14008fe112', 'ResponseMetadata': {'RequestId': '2d47728a-1b0c-4c7c-8387-e42f26fc1a90', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '2d47728a-1b0c-4c7c-8387-e42f26fc1a90', 'content-type': 'application/x-amz-json-1.1', 'content-length': '50', 'date': 'Tue, 15 Jun 2021 20:41:07 GMT'}, 'RetryAttempts': 0}}


In [12]:
r = client.get_query_results(queryId=q['queryId'])
print(r)

{'results': [[{'field': '@timestamp', 'value': '2021-06-15 20:39:07.663'}, {'field': '@message', 'value': '{"notification":{"messageMD5Sum":"b0602eeb835905924488a4d3987bafd6","messageId":"0b63d2dc-e604-5715-9e27-8b9ff4112f5c","topicArn":"arn:aws:sns:us-east-1:427848627088:t1-dev","timestamp":"2021-06-15 20:38:45.719"},"delivery":{"deliveryId":"9e5600a5-912a-58a5-871c-59e6833f88ac","destination":"arn:aws:sqs:us-east-1:427848627088:sampleQueue-dev","providerResponse":"{\\"ErrorCode\\":\\"AccessDenied\\",\\"ErrorMessage\\":\\"Access to the resource https://sqs.us-east-1.amazonaws.com/427848627088/sampleQueue-dev is denied.\\",\\"sqsRequestId\\":\\"Unrecoverable\\"}","dwellTimeMs":41,"attempts":1,"statusCode":403},"status":"FAILURE"}'}, {'field': '@ptr', 'value': 'CnMKOgo2NDI3ODQ4NjI3MDg4OnNucy91cy1lYXN0LTEvNDI3ODQ4NjI3MDg4L3QxLWRldi9GYWlsdXJlEAcSNRoYAgX8OoolAAAAAI3rlIEABgyQ+eAAAABSIAEoj+GYi6EvMI/hmIuhLzgBQIsFSOERUJkKEAAYAQ=='}]], 'statistics': {'recordsMatched': 1.0, 'recordsScanned': 1.0

### Publish Log Queries

In [13]:
# Everything in the log
import time
nowish = int(time.time())
hour_ago = nowish - (60*60)

q = client.start_query(
    logGroupName='/aws/lambda/wrap-and-pub-dev-wrapPub',
    startTime=hour_ago,
    endTime=nowish,
    queryString='fields @timestamp, @message | sort @timestamp desc | limit 20',
    limit=123
)

print(q)

{'queryId': '96fe574f-b3f4-42cc-a572-e52594dbf940', 'ResponseMetadata': {'RequestId': 'f90df3bf-a727-4043-946d-5cac28e7c889', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': 'f90df3bf-a727-4043-946d-5cac28e7c889', 'content-type': 'application/x-amz-json-1.1', 'content-length': '50', 'date': 'Tue, 15 Jun 2021 20:52:59 GMT'}, 'RetryAttempts': 0}}


In [19]:
# Just the correlations
import time
nowish = int(time.time())
hour_ago = nowish - (15*60)

q = client.start_query(
    logGroupName='/aws/lambda/wrap-and-pub-dev-wrapPub',
    startTime=hour_ago,
    endTime=nowish,
    queryString='filter @message like /PublishContext/ | fields @timestamp, @message | sort @timestamp desc | limit 20',
    limit=123
)

print(q)

{'queryId': 'eb8e7f60-2c96-4596-8923-f2f9144f37dc', 'ResponseMetadata': {'RequestId': '8fd5defc-6304-4615-9ce7-cb0dfbb2f19b', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '8fd5defc-6304-4615-9ce7-cb0dfbb2f19b', 'content-type': 'application/x-amz-json-1.1', 'content-length': '50', 'date': 'Tue, 15 Jun 2021 21:01:47 GMT'}, 'RetryAttempts': 0}}


In [20]:
r = client.get_query_results(queryId=q['queryId'])
print(r)

{'results': [[{'field': '@timestamp', 'value': '2021-06-15 20:49:29.648'}, {'field': '@message', 'value': '2021-06-15T20:49:29.648Z\ta501f020-ee06-45cc-84ec-1dd33e4b1c3a\tINFO\t{"PublishContext":{"EventId":"b13e360a-1978-4afa-8ad2-835668d22e87","MessageId":"80fbef47-d598-5822-871e-b604c1018c44"}}\n'}, {'field': '@ptr', 'value': 'Cm4KNQoxNDI3ODQ4NjI3MDg4Oi9hd3MvbGFtYmRhL3dyYXAtYW5kLXB1Yi1kZXYtd3JhcFB1YhACEjUaGAIF9o+xhQAAAAIAXzYiAAYMkSTgAAABUiABKL3WvouhLzCx3L6LoS84EEC4Q0iyWlDWNBANGAE='}], [{'field': '@timestamp', 'value': '2021-06-15 20:49:28.939'}, {'field': '@message', 'value': '2021-06-15T20:49:28.938Z\t3d1ec3f6-7c28-4d99-b247-08d99bf706b2\tINFO\t{"PublishContext":{"EventId":"b9122b09-e2f5-4ee7-8404-743837fa44d7","MessageId":"9a7d0418-9669-54b5-9c09-5d5b82f1370a"}}\n'}, {'field': '@ptr', 'value': 'Cm4KNQoxNDI3ODQ4NjI3MDg4Oi9hd3MvbGFtYmRhL3dyYXAtYW5kLXB1Yi1kZXYtd3JhcFB1YhACEjUaGAIF9o+xhQAAAAIAXzYiAAYMkSTgAAABUiABKL3WvouhLzCx3L6LoS84EEC4Q0iyWlDWNBAFGAE='}]], 'statistics': {'recordsMatch

### Clean Up

In [ ]:
try:
    sqs.delete_queue(
        QueueUrl='https://queue.amazonaws.com/' + account_no + '/sampleQueue'
    )
    print('queue deleted')
except botocore.exceptions.ClientError as error:
    if error.response['Error']['Code'] == 'AWS.SimpleQueueService.NonExistentQueue':
        print('queue deleted')
    else:
        raise error